In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from theano import *
from lasagne.layers import EmbeddingLayer, InputLayer, get_output
import lasagne
import lasagne.layers
import theano.tensor as T
import theano
import numpy as np
from helpers import SimpleMaxingLayer

Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
import json

In [4]:
with open('../external-wiki1.json') as f:
    queries = json.load(f)['queries']

In [5]:
len(queries)

9915

In [6]:
sum([any([g['gold'] for g in v.values()]) for v in queries.values()])

8917

In [7]:
8917/9915.

0.8993444276348966

In [8]:
from wordvecs import WordVectors, EmbeddingLayer

wordvectors = WordVectors(fname="../GoogleNews-vectors-negative300.bin", negvectors=False)

In [9]:
class EntityVectorLinkExp(object):
    
    def __init__(self, wordvec=wordvectors, queries=queries):
        self.wordvecs = wordvec
        self.queries = queries
        self.num_words_to_use_conv = self.wordvecs.num_words_to_use
        
        self._setup()
    
    def _setup(self):
        self.x_document_input = T.matrix('x_sent')
        self.x_surface_text_input = T.matrix('x_surface')
        self.x_target_input = T.matrix('x_target')
        self.y_score = T.matrix('y')
        
        self.embedding_W = theano.shared(self.wordvecs.get_numpy_matrix())
        
        self.document_l = lasagne.layers.InputLayer((None,self.wordvecs.num_words_to_use), input_var=self.x_document_input)
    
        self.document_embedding_l = EmbeddingLayer(
            self.document_l,
            W=self.embedding_W,
            add_word_params=False,
        )
        
        self.document_conv1_l = lasagne.layers.Conv2DLayer(
            self.document_embedding_l,
            num_filters=150,
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            name='document_conv1',
            nonlinearity=lasagne.nonlinearities.rectify,
        )
        
        self.document_max_l = lasagne.layers.Pool2DLayer(
            
        )
        